In [ ]:
# auto format code at cell execution
%load_ext lab_black

root_location = "../../"
import sys

sys.path.append(root_location)

import matplotlib.pyplot as plt
import shapely.geometry as geometry

from src.tools.pre_processing import read_csv_as_geopandas
from src.tools.visualising import (
    determine_extrema_with_border,
    load_or_get_belgium_roads,
)

# Parameters

In [ ]:
input_file_name = root_location + "data/samples/year_2018_month_12_country_Belgium.csv"

# Load data

In [ ]:
%%time
gdf = read_csv_as_geopandas(input_file_name)

In [ ]:
gdf.head()

# Backup geometry data

In [ ]:
gdf["point"] = gdf.geometry

# Generate a context to each data point

In [ ]:
%%time

context_size = 25

for i in range(1, context_size):
    gdf["next-" + str(i)] = gdf.point.shift(i)
    
gdf.dropna(inplace=True)
gdf["geometry"] = gdf.apply(
    lambda row: geometry.LineString(
        [
            row["next-" + str(i)] 
            for i 
            in range(1, context_size)
        ]
    )
    , axis=1
)

gdf.drop(
    [
        "next-" + str(i)
        for i 
        in range(1, context_size)
    ],
    axis=1,
    inplace=True
)

In [ ]:
gdf.head()

# Visualize 1 example

In [ ]:
%%time

partial_gdf = gdf.sample(1, random_state=0)

x_max_with_border, x_min_with_border, y_max_with_border, y_min_with_border = (
    determine_extrema_with_border(partial_gdf, absolute_border_size=0.003)
)

ax = load_or_get_belgium_roads(repository_root_location=root_location).plot(edgecolor="gray", figsize=(10, 6), zorder=-1)
partial_gdf.plot(ax=ax, edgecolor="red", zorder=0, linewidth=5)

plt.xlim([x_min_with_border, x_max_with_border])
plt.ylim([y_min_with_border, y_max_with_border])

partial_gdf.geometry